In [1]:
import numpy as np

NB_SIM = 5
QUANTILES = np.linspace(0.85, 0.997, 20)
print QUANTILES

[ 0.85        0.85773684  0.86547368  0.87321053  0.88094737  0.88868421
  0.89642105  0.90415789  0.91189474  0.91963158  0.92736842  0.93510526
  0.94284211  0.95057895  0.95831579  0.96605263  0.97378947  0.98152632
  0.98926316  0.997     ]


In [2]:
import sys

sys.path.append('..')

In [3]:
import os

In [4]:
def rec_dict_print(a_dict, level=1):
    keys = a_dict.keys()
    keys.sort()
    for k in keys:
        if type(a_dict[k]) is dict:
            print "".join(["-" for i in range(level)]) + k
            rec_dict_print(a_dict[k], level + 2)
        else:
            print "".join(["-" for i in range(level)]) + k

In [5]:
%matplotlib inline
import dill

loaded_data = None

with open('precomputed_sims/data%i.pkl' % (NB_SIM), 'rb') as f:
    loaded_data = dill.load(f)

In [6]:
rec_dict_print(loaded_data)

-N
-credit
---bc_ids
---bc_subsets_indexes
---copula
---default_times
-underlyings


In [7]:
N = loaded_data["N"]

In [8]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1 / 360.
delta = 5 * step

maturity = 5.

print "Maturity = %s years" % maturity

Maturity = 5.0 years


# Discount

In [9]:
from finance.discountfactor import ConstantRateDiscountFactor 

r = 0.02
discount = ConstantRateDiscountFactor(r)

# Underlying

In [10]:
udlyings = loaded_data["underlyings"]

print "Maximum number of paths: %i" % len(udlyings)

gbm0 = udlyings[0]

kappa = gbm0.drifts[0][0]
sigma = gbm0.vols[0][0]
print "kappa = %s, sigma = %s" % (kappa, sigma)

time_grid = gbm0.time

Maximum number of paths: 20000
kappa = 0.12, sigma = 0.2


# Derivative

In [11]:
derivatives_nb = 1

In [12]:
from finance.products.european.swap import (
    SwapContract,
)

swap_delta = 0.25

swap_dates = SwapContract.generate_payment_dates(0, maturity, swap_delta)
swap = SwapContract(gbm0, discount, swap_dates)

price_0 = swap.price(0., incl_next_coupon=True)

print swap
print "\nPrice swap at t=0 without 1st coupon = %s" % price_0

Swap contract of maturity T = 5 years, over S^0 with strike K = 134.306, paying at {0.00, 0.25, 0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 2.00, 2.25, 2.50, 2.75, 3.00, 3.25, 3.50, 3.75, 4.00, 4.25, 4.50, 4.75, 5.00}

Price swap at t=0 without 1st coupon = 0.0


In [13]:
p_fixed = 1.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike*delta_beta_sum)

print "Notional on the swap: %s" % notional

Notional on the swap: 0.0015687485053


# Indexes stuffs

In [14]:
copula = loaded_data["credit"]["copula"]

c_subsets_indexes = loaded_data["credit"]["bc_subsets_indexes"]

obligors_nb = len(copula.subsets[c_subsets_indexes[-1]][0])
print "Obligor numbers: %s" % obligors_nb

c_ids = [17, 9, 29, 26, 50, 4, 5, 13, 64]
c_positions = [0.69, -0.46, -0.44, -0.36, 0.34, 0.23, 0.09, -0.05, -0.04]

print "Counterparties id: %s (nb = %s)" % (c_ids, len(c_ids))

Obligor numbers: 125
Counterparties id: [17, 9, 29, 26, 50, 4, 5, 13, 64] (nb = 9)


In [15]:
positions = np.zeros(obligors_nb)
for idx, ps in zip(c_ids, c_positions):
    positions[idx] = ps

positions = positions / -positions[13]
positions = np.array(positions).reshape(positions.size, 1)

print positions.flatten()

[  0.    0.    0.    0.    4.6   1.8   0.    0.    0.   -9.2   0.    0.
   0.   -1.    0.    0.    0.   13.8   0.    0.    0.    0.    0.    0.
   0.    0.   -7.2   0.    0.   -8.8   0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    6.8   0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.   -0.8   0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0. ]


## Default times

In [16]:
default_times_mat = loaded_data["credit"]["default_times"]

In [17]:
import pandas as pd

def get_default_times_sim(c_ids_, N_, default_times_mat_, copula_, subsets_indexes_):
    matrix_def_ = np.zeros((len(c_ids_), N_))
    
    for j_, c_id_ in enumerate(c_ids_):
        cp_subsets_indexes_ = copula_.get_indexes_including(c_id_)
        c_df_times_indexes_ = [ii_ for ii_, ind_ in enumerate(subsets_indexes_) if ind_ in cp_subsets_indexes_]
        
        c_df_times_mat_ = default_times_mat_[:, c_df_times_indexes_]
        c_df_times_ = c_df_times_mat_.min(axis=1)
        
        matrix_def_[j_] = c_df_times_
        
    matrix_def_[matrix_def_==1000.] = np.nan
        
    df_default_times_ = pd.DataFrame(matrix_def_, index=np.array(c_ids_))
    return df_default_times_

In [18]:
default_times = get_default_times_sim(c_ids, N, default_times_mat, copula, c_subsets_indexes)
default_times

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,0.336111,NaN,NaN,NaN,NaN,NaN,NaN,0.388889,...,1.808333,NaN,NaN,4.041667,NaN,NaN,NaN,NaN,NaN,NaN
26,0.941667,NaN,0.677778,NaN,0.686111,NaN,NaN,NaN,2.119444,1.347222,...,0.755556,NaN,0.875,1.172222,2.552778,0.469444,NaN,0.894444,0.702778,NaN
50,NaN,1.597222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.530556
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.638889,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,NaN,0.152778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.172222,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
from utils import time_offseter

In [20]:
times_cva = np.arange(0, maturity, 0.5)

shifted_times_cva = times_cva + 1.
shifted_times_cva[-1] = maturity

#times_cva = [0.]
#shifted_times_cva = [1.]

default_times.index.values

array([17,  9, 29, 26, 50,  4,  5, 13, 64], dtype=int64)

In [21]:
from functools import partial

def fact_(swap_, discount_, kappa_, delta_, t_):
    time_grid_ = swap_.underlying.time
    t_delta_ = time_offseter(t_ + delta_, time_grid_, True)
    
    coupon_dates_ = swap_.pillars
    l_t_delta_ = np.searchsorted(coupon_dates_, t_delta_, side='left')

    beta_T_l_ = map(discount_, coupon_dates_[l_t_delta_ + 1 :])    
    h_l_ = swap_.delta_time[l_t_delta_ : ]
    
    T_l_m1_ = kappa_ * coupon_dates_[l_t_delta_ : -1]
    exp_factor_ = map(np.exp, T_l_m1_)
    
    tmp_ = np.multiply(exp_factor_, h_l_)    
    res_ = np.dot(beta_T_l_, tmp_)
    
    return res_

fact = partial(fact_, swap, discount, kappa, delta)

In [22]:
from scipy.stats import norm

def B_(fact_f_, vol_, delta_, a_, omega_, t_):
    if a_ <= 0.5: 
        a_ = 1. - a_
    
    perc_ = a_ if omega_ <= 0. else (1. - a_)
    q_ = norm.ppf(perc_)
    
    var_ = vol_**2 * delta_
    exp_factor_ = np.exp(-0.5 * var_ + np.sqrt(var_) * q_)
    
    tmp_res_ = 1. - exp_factor_
    
    return np.sign(omega_) * tmp_res_ * fact_f_(t_)

B = partial(B_, fact, sigma, delta)

In [23]:
def compute_disc_margins_(b_f_, kappa_, notional_, positions_, times_cva_, shifted_times_cva_, default_times_, udls_, q_):
    margins_ = dict()
    
    N_ = len(udls_)
    
    c_ids_ = default_times_.index.values
    
    print q_
    for t0_, t1_ in zip(times_cva_, shifted_times_cva_):
        key_ = "[%.2f, %.2f]" % (t0_, t1_)
        print key_
        margins_[key_] = pd.DataFrame(0., index=range(N_), columns=c_ids_)
        
        for j_ in xrange(N_):
            udl_ = udls_[j_]
            time_grid_ = udl_.time
            tau_j_ = default_times_.loc[c_ids_, j_]
            
            c_defaulted_ = tau_j_[(tau_j_ > t0_) & (tau_j_ <= t1_)]
            for (c_idx_, tau_) in c_defaulted_.iteritems():
                tau_ = time_offseter(tau_, time_grid_, True)
                                
                hat_s_tau_ = udl_(tau_)[0][0] * np.exp(-kappa_ * tau_)
                
                nom_i = notional_ * np.abs(positions_[c_idx_][0])
                margin_ = nom_i * hat_s_tau_ * b_f_(q_, positions_[c_idx_][0], tau_)
            
                margins_[key_].loc[j_, c_idx_] = margin_

    return margins_

compute_disc_margins = partial(compute_disc_margins_, B, kappa, notional, positions)

In [24]:
im_path = './res/sim%i/im/' % (NB_SIM)

for q in QUANTILES:
    im_q_path = os.path.join(im_path, str(q))
    if not os.path.isdir(im_q_path):
        os.makedirs(im_q_path)
        
    q_margins = compute_disc_margins(times_cva, shifted_times_cva, default_times, udlyings, q)
    for k, dataframe_ in q_margins.iteritems():
        path = os.path.join(im_q_path, 'im%s.csv' % k)
        dataframe_.to_csv(path)

0.85
[0.00, 1.00]
[0.50, 1.50]
[1.00, 2.00]
[1.50, 2.50]
[2.00, 3.00]
[2.50, 3.50]
[3.00, 4.00]
[3.50, 4.50]
[4.00, 5.00]
[4.50, 5.00]
0.857736842105
[0.00, 1.00]
[0.50, 1.50]
[1.00, 2.00]
[1.50, 2.50]
[2.00, 3.00]
[2.50, 3.50]
[3.00, 4.00]
[3.50, 4.50]
[4.00, 5.00]
[4.50, 5.00]
0.865473684211
[0.00, 1.00]
[0.50, 1.50]
[1.00, 2.00]
[1.50, 2.50]
[2.00, 3.00]
[2.50, 3.50]
[3.00, 4.00]
[3.50, 4.50]
[4.00, 5.00]
[4.50, 5.00]
0.873210526316
[0.00, 1.00]
[0.50, 1.50]
[1.00, 2.00]
[1.50, 2.50]
[2.00, 3.00]
[2.50, 3.50]
[3.00, 4.00]
[3.50, 4.50]
[4.00, 5.00]
[4.50, 5.00]
0.880947368421
[0.00, 1.00]
[0.50, 1.50]
[1.00, 2.00]
[1.50, 2.50]
[2.00, 3.00]
[2.50, 3.50]
[3.00, 4.00]
[3.50, 4.50]
[4.00, 5.00]
[4.50, 5.00]
0.888684210526
[0.00, 1.00]
[0.50, 1.50]
[1.00, 2.00]
[1.50, 2.50]
[2.00, 3.00]
[2.50, 3.50]
[3.00, 4.00]
[3.50, 4.50]
[4.00, 5.00]
[4.50, 5.00]
0.896421052632
[0.00, 1.00]
[0.50, 1.50]
[1.00, 2.00]
[1.50, 2.50]
[2.00, 3.00]
[2.50, 3.50]
[3.00, 4.00]
[3.50, 4.50]
[4.00, 5.00]
[4.50, 5